In [1]:
# pip install pymysql

In [2]:
# pip install subprocess

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement subprocess (from versions: none)
ERROR: No matching distribution found for subprocess


In [3]:
#Importing the libraries
import requests
import pandas as pd
import json 
import sqlalchemy as db
from dotenv import load_dotenv
from datetime import date
import os
import toml 
import subprocess
import boto3

In [4]:

def sql_connect(end, user, password, port,schema):
    
    engine = db.create_engine(f'mysql+pymysql://{user}:{password}@{end}:{port}/{schema}')
    return engine

connection = sql_connect('lambda-pr.crsu08qay9nu.ap-south-1.rds.amazonaws.com', 'admin', '12345678', 3306, 'superstore')


    
#Creating a the most purchase from customers 
sql =  'SELECT CustomerID, ProductID, SUM(Sales) AS most_pur FROM orders GROUP BY CustomerID ORDER BY most_pur DESC LIMIT 10;'
        

#reading the sql 
df = pd.read_sql(sql, con=connection)

In [5]:
#Reading the dataframe
df

,CustomerID,ProductID,most_pur
0,39007820,361951,97011.1940
1,38079848,373440,70045.9700
2,107565533,126813,67285.1320
3,104752832,559119,61350.0900
4,23905406,464128,56105.7305
5,91977284,710507,55692.7600
6,38075002,436896,54685.8000
7,91811409,789601,54630.1045
8,59254130,560855,54368.9085
9,64444841,567704,50131.9595


In [6]:
#Saving the dataframe from sql to a json file
df.to_json('result.json')

In [7]:
df[["CustomerID"]].head()

,CustomerID
0,39007820
1,38079848
2,107565533
3,104752832
4,23905406


In [8]:

df[["CustomerID"]].to_json('cus_id.json')

In [12]:
bucket='lambda-pr'
folder='input'


subprocess.call(['aws','s3','cp','cus_id.json', 's3://lambda-pr/input/cus_id.json'])


# s3://lambda-pr/input/cus_id.json

FileNotFoundError: [WinError 2] The system cannot find the file specified

In [14]:
def get_customer_id(cusid_file):
    """
    Read customer_id_file and return a string list of customer_id
    """
    li=[]
    with open(cusid_file,'rb') as f:
        data = json.load(f)
        for value in data["CustomerID"].values():
            li.append(str(value))
        
    id_string = "("+ ",".join(li)+")"
    return id_string

def get_customer_data(engine, sql):
    """
    Get customer data from database
    """
    mysql_result = engine.execute(sql)
    records=[]
    for row in mysql_result:
        doc={"id":row[0], "name":row[1], "date":date.today().strftime('%Y-%m-%d')}
        records.append(doc)
    data=json.dumps(records)
    return data





In [ ]:
def lambda_handler(event, context):
    print("Received event: " + json.dumps(event, indent=2))
    
    app_config = toml.load('config.toml')
    
    api_url = app_config['api']['api_url']
    
    host=app_config['db']['host']
    port=app_config['db']['port']
    database=app_config['db']['database']
    schema=app_config['db']['schema']


    load_dotenv()
    user=os.getenv('user')
    password=os.getenv('password')
    access_key=os.getenv('access_key')
    secret_key=os.getenv('secret_key')

    # read customer_id_file to local
    
    s3_client = boto3.client('s3', aws_access_key_id=access_key, aws_secret_access_key=secret_key)
    
    
    bucket = event['Records'][0]['s3']['bucket']['name']
    key = event['Records'][0]['s3']['object']['key'].split("/")[1]
    
    
    filepath = "/tmp/" +  key
    download_key = 'input/'+key
    
    s3_client.download_file(Bucket=bucket, Key= download_key, Filename=filepath)
    


    # build the connection to the database
    engine = mysql_connect(host, user, password, database, port,schema)
    
    # get the customer id and insert it into the sql statement
    ids = get_customer_id(filepath)

    sql=f"""select CustomerID, CustomerName
            from customers
            where CustomerID in {ids} ;
         """
    # print(engine)
    # get the customer data from database
        ### method 1 execute to get result
    data = get_customer_data(engine, sql)

    #     # method 2 use pandas to_json to get result
    # # data = get_customer_data_pandas(engine, sql)

    # post data to api
    request = requests.post(api_url, data=data)
    
    
    return request.status_code
